In [ ]:
import pandas as pd 
import numpy as np
import keras

In [ ]:
df = pd.read_csv(<PATH>, sep=';', engine='python')

# Feature Engineering

Create groups for resources and activities

In [ ]:
Grinding = ['Machine 10 - Grinding', 'Machine 11 - Grinding', 'Machine 12 - Grinding', 'Machine 19 - Grinding',
            'Machine 25 - Grinding', 'Machine 27 - Grinding']

Round_Grinding = ['Machine 2 - Round Grinding', 'Machine 3 - Round Grinding']

Turning_and_Milling = ['Machine 4 - Turning & Milling', 'Machine 5 - Turning & Milling', 'Machine 6 - Turning & Milling',
                       'Machine 8 - Turning & Milling', 'Machine 9 - Turning & Milling']

Turning = ['Machine 15 - Turning', 'Machine 21 - Turning', 'Machine 23 - Turning']

Milling = ['Machine 14 - Milling', 'Machine 16 - Milling', 'Machine 22 - Milling']

Quality_Check = ['Quality Check 1', 'Quality Check 2']

Wire_Cut = ['Machine 18 - Wire Cutting', 'Wire Cut - Machine 13']

In [ ]:
GrindingRework = ['Grinding Rework','Grinding Rework - Machine 12', 'Grinding Rework - Machine 2', 
                  'Grinding Rework - Machine 27']

Milling = ['Milling - Machine 10','Milling - Machine 14','Milling - Machine 16', 'Milling - Machine 8']

FlatGrinding = ['Flat Grinding - Machine 11','Flat Grinding - Machine 26']

RoundGrinding = ['Round Grinding - Machine 12', 'Round Grinding - Machine 19', 'Round Grinding - Machine 2',
                    'Round Grinding - Machine 23', 'Round Grinding - Machine 3', 'Round Grinding - Manual']

TurningMilling = ['Turning & Milling - Machine 10', 'Turning & Milling - Machine 10', 
                      'Turning & Milling - Machine 4', 'Turning & Milling - Machine 5', 
                      'Turning & Milling - Machine 6', 'Turning & Milling - Machine 8', 
                     'Turning & Milling - Machine 9']

SetupTurnMill = ['SETUP     Turning & Milling - Machine 5', 'Setup - Machine 8', 'Setup - Machine 4']

Turning = ['Turning Machine 21', 'Turning - Machine 4', 'Turning - Machine 5', 'Turning - Machine 8',
               'Turning - Machine 9']

TurnMillScrew = ['Turn & Mill. & Screw Assem - Machine 9', 'Turn & Mill. & Screw Assem - Machine 10']

WireCut = ['Wire Cut - Machine 13', 'Wire Cut - Machine 18']

Define feature engineering functions

In [ ]:
from sklearn import preprocessing


def fill_nan(df):
    
    df.fillna('N', inplace=True)
    
    return df


def group_activities(df):
    
    df['Activity'] = np.where(df['Activity'].isin(GrindingRework), 'GrindingRework', df['Activity'])
    df['Activity'] = np.where(df['Activity'].isin(Milling), 'Milling', df['Activity'])
    df['Activity'] = np.where(df['Activity'].isin(FlatGrinding), 'FlatGrinding', df['Activity'])
    df['Activity'] = np.where(df['Activity'].isin(RoundGrinding), 'RoundGrinding', df['Activity'])
    df['Activity'] = np.where(df['Activity'].isin(TurningMilling), 'TurningMilling', df['Activity'])
    df['Activity'] = np.where(df['Activity'].isin(SetupTurnMill), 'SetupTurnMill', df['Activity'])
    df['Activity'] = np.where(df['Activity'].isin(Turning), 'Turning', df['Activity'])
    df['Activity'] = np.where(df['Activity'].isin(TurnMillScrew), 'TurnMillScrew', df['Activity'])
    df['Activity'] = np.where(df['Activity'].isin(WireCut), 'WireCut', df['Activity'])
            
    return df


def group_resources(df):
    
    df['Resource'] = np.where(df['Resource'].isin(Grinding), 'Grinding', df['Resource'])
    df['Resource'] = np.where(df['Resource'].isin(Milling), 'Milling', df['Resource'])
    df['Resource'] = np.where(df['Resource'].isin(RoundGrinding), 'RoundGrinding', df['Resource'])
    df['Resource'] = np.where(df['Resource'].isin(Turning_and_Milling), 'Turning_and_Milling', df['Resource'])
    df['Resource'] = np.where(df['Resource'].isin(Turning), 'Turning', df['Resource'])
    df['Resource'] = np.where(df['Resource'].isin(Quality_Check), 'Quality_Check', df['Resource'])
    df['Resource'] = np.where(df['Resource'].isin(Wire_Cut), 'Wire_Cut', df['Resource'])
            
    return df


def get_weekday_time(df):
    
    df['Start Timestamp'] =  pd.to_datetime(df['Start Timestamp'])
    df['Complete Timestamp'] =  pd.to_datetime(df['Complete Timestamp'])
    df['Start_Weekday'] = df['Start Timestamp'].dt.dayofweek
    df['Start_Time[h]'] = df['Start Timestamp'].dt.hour
    df['Start_Time[m]'] = df['Start Timestamp'].dt.minute
    df['Start_Time[s]'] = df['Start_Time[h]'] * 3600 + df['Start_Time[m]'] * 60
    df.drop(['Start_Time[h]', 'Start_Time[m]'], axis=1, inplace=True)
    
    return df


def get_avg_rem_leadtime(df):
    
    df_copy = df.copy()
    df_copy.drop_duplicates(subset='Case ID', keep='first', inplace=True)
    df_copy = df_copy[['Part Desc.', 'RemainingTime[s]']]
    df_copy = df_copy.groupby('Part Desc.').median().reset_index()
    df_copy.rename(columns={'RemainingTime[s]': 'Avg_LeadTime[s]'}, inplace=True)
    df = df.merge(df_copy, left_on='Part Desc.', right_on = 'Part Desc.', how='left')
    
    df['Avg_Rem_LeadTime[s]'] = df['Avg_LeadTime[s]'] - df['ElapsedTime[s]']
    df.drop('Avg_LeadTime[s]', axis=1, inplace=True)
    
    return df

    
def group_and_get_qty_finished(df):
    
    df['qty_fin'] = df.groupby(['Case ID', 'Activity'])['Qty Completed'].transform(pd.Series.cumsum)
    df.drop_duplicates(subset=['Case ID', 'Activity'], keep='last', inplace=True)
    
    return df


def get_rem_act(df):
    
    df_copy = df.groupby('Case ID').size().reset_index(name='count')
    df = df.merge(df_copy, left_on=['Case ID'], right_on = ['Case ID'], how='left')
    df['sum_up'] = 1
    df['n_kum'] = df.groupby('Case ID')['sum_up'].transform(pd.Series.cumsum)
    df['rem_act'] = df['count'] - df['n_kum']
    df['%_finished'] = df['n_kum'] / df['count']
    df.drop(['count', 'sum_up', 'n_kum'], axis=1, inplace=True)
    
    return df


def get_finished(df):
    
    df['finished'] = 0
    df['finished'] = np.where(df['RemainingTime[s]'] == 0.0, 1, df['finished'])
    
    return df


def label_encode(df, cols):
    
    df[cols] = df[cols].apply(preprocessing.LabelEncoder().fit_transform)
    
    return df


def get_prefixes(df):
    
    df['Resource_Prefix'] = [y.Resource.tolist()[:z+1] for x, y in df.groupby('Case ID')for z in range(len(y))]
    df['Activity_Prefix'] = [y.Activity.tolist()[:z+1] for x, y in df.groupby('Case ID')for z in range(len(y))]

    return df


def transform_span(df):
    
    df['Span'] = df.apply(lambda x: x['Span'][1:], axis = 1)
    df['Span'] = [x + ':00' for x in df['Span']]
    df['Span'] = pd.to_timedelta(df['Span'])
    df['Span'] = df['Span'].dt.total_seconds()
    df.rename(columns={'Span': 'Span[s]'}, inplace=True)
      
    return df

Compute Feature Engineering

In [ ]:
df2 = fill_nan(df)
df3 = group_activities(df2)
df4 = group_resources(df3)
df5 = get_weekday_time(df4)
df6 = get_avg_rem_leadtime(df5)
df7 = group_and_get_qty_finished(df6)
df8 = get_rem_act(df7)
df9 = get_finished(df8)
df10 = label_encode(df9, ['Activity', 'Resource', 'Part Desc.'])
df11 = get_prefixes(df10)

# Preprocessing

Define Preprocessing functions

In [ ]:
def rearrange_order(df, alg):
    
    if alg == 'ANN':
        df = df[['RemainingTime[s]', 'rem_act', '%_finished', 'ElapsedTime[s]', 'Avg_Rem_LeadTime[s]', 
                 'finished', 'Case ID']]
        
    elif alg == 'XGB':
        df = df[['rem_DLZ_[m]', 'elapsed_time_compl[m]', 'rem_meth', 'rem_act', '%_finished', 
                 'rem_DLZ_[m]_avg', 'Spl_ID']]


def set_index(df, idx_col):
    
    df.set_index(idx_col, inplace=True)
    
    return df


def get_sw_cols(df):
    
    cols = df.columns
    num_cols = df._get_numeric_data().columns
    not_sw_cols = set(cols) - set(num_cols)
    sw_cols = set(cols) - set(not_sw_cols)
    
    return sw_cols 


def sliding_window(df, cols, w):
    
    for i in range(1, w+1):  
        for ele in cols:
            df[ele + '(t-' + str(i) + ')'] = df.groupby('Case ID')[ele].shift(i)
    
    df.fillna(0, inplace=True)
    df.drop('Case ID', axis=1, inplace=True)

    df.drop('RemainingTime[s](t-1)', axis=1, inplace=True)
    df.drop('RemainingTime[s](t-2)', axis=1, inplace=True)
    
    return df


def split_data(df, train_size, alg):
    
    if alg == 'NN':
        train = df.iloc[:train_size]
        test = df.iloc[train_size:]
        train_vals = train.values
        test_vals = test.values
        return train_vals, test_vals 
    
    if alg == 'XGBoost':
        train = df.iloc[:train_size]
        test = df.iloc[train_size:]
        return train, test
    

def scale(train, test, srange):
    
    scaler = MinMaxScaler(feature_range=srange)
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.fit_transform(test)
    
    return scaler, train_scaled, test_scaled


def reshape_input(train, test, alg):
    
    if alg == 'ANN':
        X, y = train[:, 1:], train[:, 0]
        X_test, y_test = test[:, 1:], test[:, 0]
        return X, y, X_test, y_test
    
    elif alg == 'XGBoost':
        X, y = train.iloc[:, 1:], train.iloc[:, 0]
        X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]
        return X, y, X_test, y_test
    
    elif alg == 'LSTM':
        X, y = train_scaled[:, 1:], train_scaled[:, 0]
        X = X.reshape(X.shape[0], 1, X.shape[1])
        X_test, y_test = test_scaled[:, 1:], test_scaled[:, 0]
        X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
        return X, y, X_test, y_test

# Artificial Neural Network

Compute Preprocessing for ANN Model

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#Create input matrix for numerical and label-encoded values
df_small_ann = df11.drop(['Start Timestamp', 'Work Order  Qty', 'Worker ID', 'Report Type', 
                        'Qty Completed', 'Qty Rejected', 'Qty for MRB', 'Rework', 'Resource_Prefix', 
                        'Activity_Prefix', 'Span', 'Complete Timestamp', 'qty_fin', 
                          'Start_Weekday', 'Activity', 'Resource', 'Part Desc.', 'Start_Time[s]'], axis = 1)
sw_cols = get_sw_cols(df_small_ann)
df_small_ann = sliding_window(df_small_ann, sw_cols, 2)                                    
train_vals, test_vals = split_data(df_small_ann, 1092, 'NN')
ann_scaler, train_scaled, test_scaled = scale(train_vals, test_vals, (0, 1))
X, y, X_test, y_test = reshape_input(train_scaled, test_scaled, 'ANN')

Define Functions for Model building, training and testing

In [ ]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error, mean_absolute_error
from numpy import concatenate
from math import sqrt


# =========================== Build Model ===========================

def build_ann_model(model_type, neurons):
    
    if model_type == 'one_hidden_layer':    
        model = keras.models.Sequential([
            keras.layers.Dense(17, activation='relu', input_shape=[X.shape[1]]),
            keras.layers.Dense(neurons, activation='relu'),
            keras.layers.Dense(1)])
        
        model.compile(loss='mean_absolute_error',
                optimizer=tf.optimizers.Adam(learning_rate=0.0001),
                metrics=['mae', 'mse'])       
        return model
    
    elif model_type == 'two_hidden_layers':
        model = keras.models.Sequential([
            keras.layers.Dense(17, activation='relu', input_shape=[X.shape[1]]),
            keras.layers.Dense(neurons, activation='relu'),
            keras.layers.Dense(neurons, activation='relu'),
            keras.layers.Dense(1)])
        
        model.compile(loss='mean_absolute_error',
                optimizer=tf.optimizers.Adam(learning_rate=0.0001),
                metrics=['mae', 'mse'])         
        return model
    
    elif model_type == 'three_hidden_layers':
        model = keras.models.Sequential([            
            keras.layers.Dense(17, activation='relu', input_shape=[X.shape[1]]),
            keras.layers.Dense(neurons, activation='relu'),
            keras.layers.Dense(neurons, activation='relu'),
            keras.layers.Dense(neurons, activation='relu'),
            keras.layers.Dense(1)])
        
        model.compile(loss='mean_absolute_error',
                optimizer=tf.optimizers.Adam(learning_rate=0.0001),
                metrics=['mae', 'mse']) 
        return model
    
    elif model_type == 'four_hidden_layers':
        model = keras.models.Sequential([            
            keras.layers.Dense(17, activation='relu', input_shape=[X.shape[1]]),
            keras.layers.Dense(neurons, activation='relu'),
            keras.layers.Dense(neurons, activation='relu'),
            keras.layers.Dense(neurons, activation='relu'),
            keras.layers.Dense(neurons, activation='relu'),
            keras.layers.Dense(1)])
        
        model.compile(loss='mean_absolute_error',
                optimizer=tf.optimizers.Adam(learning_rate=0.0001),
                metrics=['mae', 'mse']) 
        return model


# =========================== Train Model ===========================

def fit_ann_model(model, X, y, X_test, y_test):
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=15)
    checkpoint_cb = ModelCheckpoint('ann_model.h5', save_best_only=True)
    
    history = model.fit(X, y, batch_size=64, epochs=150, validation_data=(X_test, y_test),
                            verbose=0, callbacks=[early_stopping, checkpoint_cb], shuffle=False)

    # plot history
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.show()
    
    return model
    

# =========================== Test Model ===========================

def test_ann_model(model, X_test, y_test, scaler):
    
    yhat = model.predict(X_test)
    X_orig = X_test.copy()
    
    # invert scaling for forecast
    inv_yhat = concatenate((yhat, X_orig), axis=1)
    inv_yhat = ann_scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]

    # invert scaling for actual
    y_test_orig = y_test.copy()
    y_test_orig = y_test_orig.reshape(len(y_test_orig), 1)
    inv_y = concatenate((y_test_orig, X_orig), axis=1)
    inv_y = ann_scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    # calculate errors
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))/3600
    mae = mean_absolute_error(inv_y, inv_yhat)/3600
    
    print('Test RMSE: %.3f' % rmse)
    print('Test MAE: %.3f' % mae)

Build Models

In [ ]:
ann_model_1 = build_ann_model('one_hidden_layer', 500)
ann_model_2 = build_ann_model('one_hidden_layer', 1000)
ann_model_3 = build_ann_model('one_hidden_layer', 1500)
ann_model_4 = build_ann_model('two_hidden_layers', 500)
ann_model_5 = build_ann_model('two_hidden_layers', 1000)
ann_model_6 = build_ann_model('two_hidden_layers', 1500)
ann_model_7 = build_ann_model('three_hidden_layers', 500)
ann_model_8 = build_ann_model('three_hidden_layers', 1000)
ann_model_9 = build_ann_model('three_hidden_layers', 1500)
ann_model_10 = build_ann_model('four_hidden_layers', 500)
ann_model_11 = build_ann_model('four_hidden_layers', 1000)
ann_model_12 = build_ann_model('four_hidden_layers', 1500)

ann_model_list = [ann_model_1, ann_model_2, ann_model_3, ann_model_4, ann_model_5, ann_model_6,
              ann_model_7, ann_model_8, ann_model_9, ann_model_10, ann_model_11, ann_model_12]

Train Models

In [ ]:
for model in ann_model_list:
        
        model = fit_ann_model(model, X, y, X_test, y_test)

Test Models

In [ ]:
for model in ann_model_list:
        
        test_ann_model(model, X_test, y_test, ann_scaler)

# Long Short-Term Memory Network

Compute Preprocessing for LSTM Model

In [ ]:
#Create input matrix for numerical and label-encoded values
df_small_lstm = df11.drop(['Start Timestamp', 'Work Order  Qty', 'Worker ID', 'Report Type', 
                        'Qty Completed', 'Qty Rejected', 'Qty for MRB', 'Rework', 'Resource_Prefix', 
                        'Activity_Prefix', 'Span', 'qty_fin', 'Start_Weekday'], axis = 1)
sw_cols = get_sw_cols(df_small_lstm)
df_small_lstm = sliding_window(df_small_lstm, sw_cols, 2)
df_small_lstm = set_index(df_small_lstm, 'Complete Timestamp')                                     
train_vals, test_vals = split_data(df_small_lstm, 1092, 'NN')
lstm_scaler, train_scaled, test_scaled = scale(train_vals, test_vals, (-1, 1))
X, y, X_test, y_test = reshape_input(train_scaled, test_scaled, 'LSTM')

Define Functions for Model building, training and testing

In [ ]:
# =========================== Build Model ===========================


def build_lstm_model(model_type, neurons):
    
    if model_type == 'one_hidden_layer':
        model = keras.models.Sequential([
            keras.layers.LSTM(neurons, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
            keras.layers.LSTM(neurons),
            keras.layers.Dense(1)])
        
        model.compile(loss='mean_absolute_error',
                optimizer=tf.optimizers.Adam(learning_rate=0.0001),
                metrics=['mae', 'mse'])
        return model
    
    elif model_type == 'two_hidden_layers':
        model = keras.models.Sequential([
            keras.layers.LSTM(neurons, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
            keras.layers.LSTM(neurons, return_sequences=True),
            keras.layers.LSTM(neurons),
            keras.layers.Dense(1)])
        
        model.compile(loss='mean_absolute_error',
                optimizer=tf.optimizers.Adam(learning_rate=0.0001),
                metrics=['mae', 'mse'])
        return model
    
    elif model_type == 'three_hidden_layers':
        model = keras.models.Sequential([
            keras.layers.LSTM(neurons, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
            keras.layers.LSTM(neurons, return_sequences=True),
            keras.layers.LSTM(neurons, return_sequences=True),
            keras.layers.LSTM(neurons),
            keras.layers.Dense(1)])
        
        model.compile(loss='mean_absolute_error',
                optimizer=tf.optimizers.Adam(learning_rate=0.0001),
                metrics=['mae', 'mse'])
        return model
    
    elif model_type == 'four_hidden_layers':
        model = keras.models.Sequential([
            keras.layers.LSTM(neurons, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
            keras.layers.LSTM(neurons, return_sequences=True),
            keras.layers.LSTM(neurons, return_sequences=True),
            keras.layers.LSTM(neurons, return_sequences=True),
            keras.layers.LSTM(neurons),
            keras.layers.Dense(1)])
        
        model.compile(loss='mean_absolute_error',
                optimizer=tf.optimizers.Adam(learning_rate=0.0001),
                metrics=['mae', 'mse'])
        return model
    

# =========================== Train Model ===========================

def fit_lstm_model(model, X, y, X_test, y_test):
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=15)
    checkpoint_cb = ModelCheckpoint('lstm_model.h5', save_best_only=True)
    
    history = model.fit(X, y, batch_size=32, epochs=150, validation_data=(X_test, y_test), 
                        verbose=0, callbacks=[early_stopping, checkpoint_cb], shuffle=False)

    # plot history
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.show()
    
    return model


# =========================== Test Model ===========================

def test_lstm_model(model, X_test, y_test, scaler):
    
    yhat = model.predict(X_test)
    X_orig = X_test.copy()
    X_orig = X_orig.reshape((X_orig.shape[0], X_orig.shape[2]))
    
    # invert scaling for forecast
    inv_yhat = concatenate((yhat, X_orig), axis=1)
    inv_yhat = lstm_scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]

    # invert scaling for actual
    y_test_orig = y_test.copy()
    y_test_orig = y_test_orig.reshape((len(y_test_orig), 1))
    inv_y = concatenate((y_test_orig, X_orig), axis=1)
    inv_y = lstm_scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    # calculate errors
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))/3600
    mae = mean_absolute_error(inv_y, inv_yhat)/3600
    
    print('Test RMSE: %.3f' % rmse)
    print('Test MAE: %.3f' % mae)

Build Models

In [ ]:
lstm_model_1 = build_lstm_model('one_hidden_layer', 100)
lstm_model_2 = build_lstm_model('one_hidden_layer', 200)
lstm_model_3 = build_lstm_model('one_hidden_layer', 300)
lstm_model_4 = build_lstm_model('two_hidden_layers', 100)
lstm_model_5 = build_lstm_model('two_hidden_layers', 200)
lstm_model_6 = build_lstm_model('two_hidden_layers', 300)
lstm_model_7 = build_lstm_model('three_hidden_layers', 100)
lstm_model_8 = build_lstm_model('three_hidden_layers', 200)
lstm_model_9 = build_lstm_model('three_hidden_layers', 300)
lstm_model_10 = build_lstm_model('four_hidden_layers', 100)
lstm_model_11 = build_lstm_model('four_hidden_layers', 200)
lstm_model_12 = build_lstm_model('four_hidden_layers', 300)

lstm_model_list = [lstm_model_1, lstm_model_2, lstm_model_3, lstm_model_4, lstm_model_5, lstm_model_6,
                   lstm_model_7, lstm_model_8, lstm_model_9, lstm_model_10, lstm_model_11, lstm_model_12]

Train Models

In [ ]:
for model in lstm_model_list:
        
        model = fit_lstm_model(model, X, y, X_test, y_test)

Test Models

In [ ]:
for model in lstm_model_list:
        
        test_lstm_model(model, X_test, y_test, lstm_scaler)

# XGBoost

Compute Preprocessing for XGBoost Model

In [ ]:
import xgboost as xgb
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

#Create input matrix for numerical and label-encoded values
df_small_xgb = df11.drop(['Start Timestamp', 'Work Order  Qty', 'Worker ID', 'Report Type', 
                        'Qty Completed', 'Qty Rejected', 'Qty for MRB', 'Rework', 'Resource_Prefix', 
                        'Activity_Prefix', 'Span', 'Complete Timestamp'], axis = 1)
sw_cols = get_sw_cols(df_small_xgb)
df_small_xgb = sliding_window(df_small_xgb, sw_cols, 2) 
df_small_xgb.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<')))
                         else col for col in df_small_xgb.columns.values]
df_small_xgb.rename(columns = lambda x: x.replace(' ', '_'), inplace=True)
train, test = split_data(df_small_xgb, 1092, 'XGBoost')
X, y, X_test, y_test = reshape_input(train, test, 'XGBoost')
train_dmatrix = xgb.DMatrix(data=X,label=y)

Define Function for Gridsearch

In [ ]:
from sklearn.model_selection import GridSearchCV


def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error'):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

Perform Gridsearch

In [ ]:
model = xgb.XGBRegressor()
param_grid = {
    'objective': ['reg:linear', 'reg:logistic'],
    'colsample_bytree': [0.3, 0.4, 0.5],
    'n_estimators': [10, 50, 100],
    'max_depth': [5, 10, 15],
    'alpha': [5, 10, 20],
    'learning_rate': [0.1, 0.01, 0.05]}

model, pred = algorithm_pipeline(X, X_test, y, y_test, model, 
                                 param_grid, cv=2)

print(np.sqrt(-model.best_score_)/3600)
print(model.best_params_)

Build Model

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                          max_depth = 5, alpha = 10, n_estimators = 10)

Train Model

In [ ]:
xg_reg.fit(X, y)

Test Model

In [ ]:
yhat_xgb = xg_reg.predict(X_test)

# calculate errors
rmse_xgb = sqrt(mean_squared_error(y_test, yhat_xgb))/3600
mae_xgb = mean_absolute_error(y_test, yhat_xgb)/3600
    
print('Deep Model Test RMSE: %.3f' % rmse_xgb)
print('Deep Model Test MAE: %.3f' % mae_xgb)